<a href="https://colab.research.google.com/github/changw121/Best-websites-a-programmer-should-visit/blob/master/pyG%E5%B0%8F%E5%AE%9E%E6%88%9801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pyG中的数据处理、数据集、加载器和转换
使用GCN层
在Cora引文数据集上实验

In [2]:
!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html


In [3]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [3]:
dataset

Cora()

In [4]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [7]:
dataset[0].y.unique() #一共多少种类别

tensor([0, 1, 2, 3, 4, 5, 6])

In [5]:
dataset.num_node_features

1433

In [6]:
dataset.num_classes #直接输出一共多少种类别

7

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16) # 输入维度是每个节点的特征数目 # 16是啥
        self.conv2 = GCNConv(16, dataset.num_classes) # 输出的是类别数

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [5]:
from torch.optim import optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) #优化器

In [10]:
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [6]:
from  sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch%10 == 0:
      print(f'epoch:{epoch}, loss:{loss}, acc:{accuracy_score(data.y[data.val_mask].cpu(), out.argmax(dim=1)[data.val_mask].cpu())}')

epoch:0, loss:1.9568045139312744, acc:0.186
epoch:10, loss:0.9536699652671814, acc:0.542
epoch:20, loss:0.3142772614955902, acc:0.718
epoch:30, loss:0.11366307735443115, acc:0.716
epoch:40, loss:0.06791705638170242, acc:0.732
epoch:50, loss:0.05593932047486305, acc:0.72
epoch:60, loss:0.04146436229348183, acc:0.744
epoch:70, loss:0.04927568882703781, acc:0.76
epoch:80, loss:0.033658694475889206, acc:0.752
epoch:90, loss:0.04601650685071945, acc:0.72


In [7]:
from  sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
model.eval()
pred = model(data).argmax(dim=1)
acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
prec = precision_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(),average="macro")
rec = recall_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(),average="macro")
f1 = f1_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(),average="macro")
print("The accuracy is: ",acc)
print("The precision is: ",prec)
print("The recall is: ",rec)
print("The f1 is: ",f1)

The accuracy is:  0.807
The precision is:  0.7927584796180814
The recall is:  0.8207062754455122
The f1 is:  0.801821797542347
